In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import skimage
import utils
import os

In [ ]:
def convolve_im_freq(im: np.array,
                fft_kernel: np.array,
                verbose=True):
    """ Convolves the image (im) with the frequency kernel (fft_kernel),
        and returns the resulting image.

        "verbose" can be used for turning on/off visualization
        convolution

    Args:
        im: np.array of shape [H, W]
        fft_kernel: np.array of shape [H, W] 
        verbose: bool
    Returns:
        im: np.array of shape [H, W]
    """
    ### START YOUR CODE HERE ### (You can change anything inside this block)
    fft = np.fft.fft2(im)
    to_visualize = np.fft.fftshift(fft)
    to_visualize = np.log(np.sqrt(to_visualize.real**2+to_visualize.imag**2)+1)
    fft_kernel_vis = np.fft.fftshift(fft_kernel)
    fft_kernel_vis = np.log(np.sqrt(fft_kernel_vis.real**2+fft_kernel_vis.imag**2)+1)

    conv = fft * fft_kernel
    
    conv_to_visualize = np.fft.fftshift(conv)
    conv_to_visualize = np.log(np.sqrt(conv_to_visualize.real**2+conv_to_visualize.imag**2)+1)
    conv_result = np.fft.ifft2(conv).real
    
    if verbose:
        # Use plt.subplot to place two or more images beside eachother
        plt.figure(figsize=(20, 4))
        # plt.subplot(num_rows, num_cols, position (1-indexed))
        plt.subplot(1, 5, 1)
        plt.imshow(im, cmap="gray")
        plt.title("Original Image")

        plt.subplot(1, 5, 2) 
        plt.imshow(to_visualize,cmap="gray")
        plt.title("FFT Image")
        
        plt.subplot(1, 5, 3)
        # Visualize FFT kernel
        plt.imshow(fft_kernel_vis,cmap="gray")
        plt.title("FFT Kernel")
        plt.subplot(1, 5, 4)
        # Visualize filtered FFT image
        plt.imshow(conv_to_visualize,cmap="gray")
        plt.title("Filtered FFT Image")
        plt.subplot(1, 5, 5) 
        # Visualize filtered spatial image
        plt.imshow(conv_result, cmap="gray")
        plt.title("Filtered Spatial Image")
        
    return conv_result

In [ ]:
im = skimage.data.camera()
im = utils.uint8_to_float(im)

frequency_kernel_low_pass = utils.create_low_pass_frequency_kernel(im, radius=50)
image_low_pass = convolve_im_freq(im, frequency_kernel_low_pass,verbose=True)

frequency_kernel_high_pass = utils.create_high_pass_frequency_kernel(im, radius=50)
image_high_pass = convolve_im_freq(im, frequency_kernel_high_pass,verbose=True)

plt.show()
utils.save_im("camera_low_pass.png", image_low_pass)
utils.save_im("camera_high_pass.png", image_high_pass)

In [ ]:
def convolve_im_spatial(im: np.array,
                kernel: np.array,
                verbose=True):
    """ Convolves the image (im) with the spatial kernel (kernel),
        and returns the resulting image.

        "verbose" can be used for turning on/off visualization
        convolution
        
        Note: kernel can be of different shape than im.

    Args:
        im: np.array of shape [H, W]
        kernel: np.array of shape [K, K] 
        verbose: bool
    Returns:
        im: np.array of shape [H, W]
    """
    orig_im = im.copy()
    orig_shape = im.shape
    conv_result = im
    fft = np.fft.fft2(im)
    to_vis = np.fft.fftshift(fft)
    to_vis = np.sqrt(to_vis.real**2+to_vis.imag**2)
    to_vis = np.log(to_vis+1)

    padded_filter = np.zeros(orig_shape)
    kernel_shape = kernel.shape
    padded_filter[:kernel_shape[0],:kernel_shape[1]] = kernel

    fft_kernel = np.fft.fft2(padded_filter)
    fft_kernel_vis = np.fft.fftshift(fft_kernel)
    fft_kernel_vis = np.sqrt(fft_kernel_vis.real**2+fft_kernel_vis.imag**2)
    fft_kernel_vis = np.log(fft_kernel_vis+1)

    conv = fft_kernel * fft

    conv_vis = np.fft.fftshift(conv)
    conv_vis = np.sqrt(conv_vis.real**2+conv_vis.imag**2)
    conv_vis = np.log(conv_vis+1)

    conv_result = np.fft.ifft2(conv).real
    
    

    if verbose:
        # Use plt.subplot to place two or more images beside eachother
        plt.figure(figsize=(20, 4))
        # plt.subplot(num_rows, num_cols, position (1-indexed))
        plt.subplot(1, 5, 1)
        plt.imshow((orig_im.astype('float')), cmap="gray")
        plt.title("Original Image")
        
        plt.subplot(1, 5, 2) 
        # Visualize FFT
        plt.imshow(to_vis,cmap="grey")
        plt.title("FFT Image")
        
        plt.subplot(1, 5, 3)
        # Visualize FFT kernel
        plt.imshow(fft_kernel_vis,cmap="gray")
        plt.title("FFT Kernel")
        
        plt.subplot(1, 5, 4)
        # Visualize filtered FFT image
        plt.imshow(conv_vis,cmap="gray")
        plt.title("Filtered FFT Image")
        
        plt.subplot(1, 5, 5) 
        # Visualize filtered spatial image
        plt.imshow(conv_result, cmap="gray")
        plt.title("Filtered Spatial Image")
        
    return conv_result

In [ ]:



# Changing this code should not be needed
im = skimage.data.camera()
im = utils.uint8_to_float(im)

gaussian_kernel = np.array([
    [1, 4, 6, 4, 1],
    [4, 16, 24, 16, 4],
    [6, 24, 36, 24, 6],
    [4, 16, 24, 16, 4],
    [1, 4, 6, 4, 1],
]) / 256
image_gaussian = convolve_im_spatial(im, gaussian_kernel, verbose=True)

sobel_horizontal = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
])
image_sobelx = convolve_im_spatial(im, sobel_horizontal, verbose=True)

plt.show()

utils.save_im("camera_gaussian.png", image_gaussian)
utils.save_im("camera_sobelx.png", image_sobelx)

In [ ]:
def create_kernel(im):
    """
    Creates a filter kernel in the frequency domain.

    Args:
        im: np.array of shape [H, W]
    Returns:
        kernel: np.array of shape [H, W]
    """

    kernel = np.ones_like(im)
    kernel[0:5,20:int(kernel.shape[1])-20] = 0
    return kernel

In [ ]:
impath = os.path.join("images", "noisy_moon.png")
im = utils.read_im(impath)

kernel = create_kernel(im)        
im_filtered = convolve_im_freq(im, kernel, True) 
    
utils.save_im("moon_filtered.png", utils.normalize(im_filtered))